In [1]:
import pandas as pd
import numpy as np
# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

### Load excel file with table and derivative variables of interest

In [2]:
# imaging
tvars = pd.read_excel('sesher_adi_deprivation_ind4-6.xlsx')
tvars.tail()

,Table,Variable,Description
0,led_l_adi,reshist_addr1_adi_edu_l,Residential history derived - Area Deprivation...
1,led_l_adi,reshist_addr1_adi_sp,Residential history derived - Area Deprivation...
2,led_l_adi,reshist_addr1_adi_unemp,Residential history derived - Area Deprivation...


In [3]:
dpath = '/home/cglab/projects/abcd/data/abcd5.1-rser/'

In [4]:
import os
from itertools import combinations

# recursively search directory and subdirectories for csv file
def findFile(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
            
def getDerivatives(df, table_file, table_key, how_merge='outer'):
    dpath = '/home/cglab/projects/abcd/data/abcd-data-release-5.1/'
    # find filepath, which is full path and name of file
    filepath = findFile(table_file, path=dpath)
    dat = pd.read_csv(filepath)
    # get table name, which is the string before the period
    table = table_file.split('.')[0]
    # get column names sub and event which will need for merging dataframes
    # it's the same for each df so overwriting is fine
    se_nms = dat.columns[:2].values.tolist()
    # derivative variables desired
    derivative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()
    derivative_cols += se_nms
    # merge with overall with INNER join bc we dont want to exclude participants who have task mri data but not resting or vice versa
    print('Prior to merge df size is {0} and other df shape is {1}\n merging...\n'.format(df.shape, dat[derivative_cols].shape))
    if 'src_subject_id' not in df.columns:
        # for first table assign it to df
        df = dat[derivative_cols].copy()
    else:
        # all others are merged
        df = df.merge(dat[derivative_cols], how=how_merge, on=['src_subject_id', 'eventname'])
    print('Any duplicated columns? {}'.format(df.columns.duplicated().any()))
    print('New df size is {}'.format(df.shape))
    return df


def dropDuplicateCols(df):
    # check every combination of columns
    # if ALL values are EQUAL, get the 2 columns
    dup_pairs = [(i, j) for i,j in combinations(df, 2) if df[i].equals(df[j])]
    cols_to_remove = []
    # print the columns that match and get 1st one for removal
    for i, j in dup_pairs:
        print('{0} is identical to {1}'.format(i.upper(), j.upper()))
        cols_to_remove.append(i)    
    # drop duplicate columns
    df.drop(columns=cols_to_remove, inplace=True)
    # return df WITHOUT duplicate columns
    return df

#check if there are any MRI variables requested
# if so, return the appropriate Quality control Variables
def anyMRIvars(df):
    # create empty list to store QC vars needed
    qc_vars = []
    # if any columns are found that startwith 'rsfmri_', append resting QC vars & the csv file they're found in,..
    # ... creating a list of tuples which will be combined and converted to dataframe below
    # this relies on the implicit boolean nature of lists, if empty their FALSE
    if [col for col in df.columns if col.startswith('rsfmri_')]:
        qc_vars += [('mri_y_qc_incl', 'imgincl_rsfmri_include'), ('mri_y_qc_motion', 'rsfmri_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]
    
    # if any columns are found that startwith 'rsfmri_', append resting QC vars & the csv file they're found in
    if [col for col in df.columns if col.startswith('tfmri_nback_') or col.startswith('tfabwdp')  or col.startswith('tnbasem')]:
        qc_vars += [('mri_y_qc_incl', 'imgincl_nback_include'), ('mri_y_qc_motion', 'tfmri_nback_all_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]
    
    # if qc_vars is NOT empty
    if qc_vars:
        # remove duplicates from qc_vars
        # calling 'set' function on qc_vars returns only unique items
        # then calling 'list' function converts it back to a list
        qc_vars = list(set(qc_vars))
        # set up table_key dataframe for pulling QC vars
        # includes all QC vars selected above with cooresponding table name
        qc_key = pd.DataFrame(qc_vars, columns=['Table', 'Variable'])
        # get list of unique QC tables from qc_key tuple ex. ('mri_y_qc_incl', 'imgincl_rsfmri_include')
        qc_tables = list(set([table for table, var in qc_vars]))
        # append QC variables to dataframe
        for table in qc_tables:
            print('\nPulling from table: {0}.csv\n'.format(table))
            df = getDerivatives(df, table + '.csv', qc_key, how_merge='left')
        # output dataframe with QC variables merged
        return df
    else:
        # else return the unchanged dataframe
        return df
        

### Get tables that will be pulled
* append '.csv' for data loading later
* exclude temporal variance tables because interpretation of them unclear

In [5]:
data_files = [t + '.csv' for t in tvars['Table'].unique() if 'var' not in t]
data_files

['led_l_adi.csv']

## Make sure that first csv that is pulled from has multiple 'eventnames', so that they're included in future merges

### Load, Select, & Merge data, 
* for every file in 'data_files'
* and every variable listed in 'tvars'
* append to rs

In [6]:
# set empty dataframe  
# to be filled with merges
rs = pd.DataFrame()

for file in data_files:
    rs = getDerivatives(rs, file, tvars)

Prior to merge df size is (0, 0) and other df shape is (11215, 5)
 merging...

Any duplicated columns? False
New df size is (11215, 5)


In [7]:
rs.loc[:, rs.columns.duplicated()==1]

""
0
1
2
3
4
...
11210
11211
11212
11213


### Drop columns with duplicate NAMES

In [8]:
rs = rs.loc[:,~rs.columns.duplicated()].copy()
print('Any duplicated columns? {}'.format(rs.columns.duplicated().any()))

Any duplicated columns? False


### Drop columns with duplicate VALUES

In [9]:
print(rs.shape)
rs = dropDuplicateCols(rs)
print(rs.shape)

(11215, 5)
(11215, 5)


In [10]:
if 'rsfmri_c_ngd_dt_ngd_dt' in rs.columns.tolist():
    print('included')

In [11]:
# check number of missing values in each column
rs.isnull().sum()

reshist_addr1_adi_edu_l    1
reshist_addr1_adi_sp       1
reshist_addr1_adi_unemp    1
src_subject_id             0
eventname                  0
dtype: int64

In [12]:
rs.eventname.value_counts()

eventname
baseline_year_1_arm_1    11215
Name: count, dtype: int64

### Check if any MRI variables present
* if so, append the appropriate ABCD Quality Control Variables
* for scanner movement, artifacts, etc

In [13]:
print(rs.shape)
rs = anyMRIvars(rs)
print(rs.shape)

(38815, 364)

Pulling from table: mri_y_adm_info.csv

Prior to merge df size is (38815, 364) and other df shape is (22939, 3)
 merging...

Any duplicated columns? False
New df size is (38815, 365)

Pulling from table: mri_y_qc_motion.csv

Prior to merge df size is (38815, 365) and other df shape is (22460, 4)
 merging...

Any duplicated columns? False
New df size is (38815, 367)

Pulling from table: mri_y_qc_incl.csv

Prior to merge df size is (38815, 367) and other df shape is (22939, 4)
 merging...

Any duplicated columns? False
New df size is (38815, 369)
(38815, 369)


In [13]:
rs['eventname'].value_counts()

eventname
baseline_year_1_arm_1    11215
Name: count, dtype: int64

### Save overall df to csv output

In [14]:
rs.shape

(11215, 5)

In [16]:
if 'rsfmri_c_ngd_dt_ngd_dt' in rs.columns.tolist():
    rs.to_csv(dpath + 'abcd5.1_rtmri_2b_epn_cort_subc_net_df_urg_dep_.csv', index=False)
else:
    rs.to_csv(dpath + 'abcd5.1_rtmri_2b_epn_cort_subc_net_urg_dep_.csv', index=False)

In [15]:
rs.to_csv(dpath + 'abcd5.1_adi_dep_ind4-6.csv', index=False)

# END

#### For troubleshooting

In [19]:
df = pd.DataFrame()
table_file = 'nc_y_nihtb.csv'
table_key = svars1
how_merge = 'outer'

dat = pd.read_csv(dpath + table_file)
# get table name, which is the string before the period
table = table_file.split('.')[0]
# get column names sub and event which will need for merging dataframes
# it's the same for each df so overwriting is fine
se_nms = [c for c in dat.columns if c in ['src_subject_id', 'eventname']]
# derivative variables desired
derivative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()
derivative_cols += se_nms

print(len(derivative_cols))
derivative_cols

68


['rsfmri_cor_ngd_sa_scs_aalh',
 'rsfmri_cor_ngd_sa_scs_aarh',
 'rsfmri_cor_ngd_sa_scs_aglh',
 'rsfmri_cor_ngd_sa_scs_agrh',
 'rsfmri_cor_ngd_sa_scs_bs',
 'rsfmri_cor_ngd_sa_scs_cdelh',
 'rsfmri_cor_ngd_sa_scs_cderh',
 'rsfmri_cor_ngd_sa_scs_crcxlh',
 'rsfmri_cor_ngd_sa_scs_crcxrh',
 'rsfmri_cor_ngd_sa_scs_hplh',
 'rsfmri_cor_ngd_sa_scs_hprh',
 'rsfmri_cor_ngd_sa_scs_pllh',
 'rsfmri_cor_ngd_sa_scs_plrh',
 'rsfmri_cor_ngd_sa_scs_ptlh',
 'rsfmri_cor_ngd_sa_scs_ptrh',
 'rsfmri_cor_ngd_sa_scs_thplh',
 'rsfmri_cor_ngd_sa_scs_thprh',
 'rsfmri_cor_ngd_sa_scs_vtdclh',
 'rsfmri_cor_ngd_sa_scs_vtdcrh',
 'rsfmri_cor_ngd_df_scs_aalh',
 'rsfmri_cor_ngd_df_scs_aarh',
 'rsfmri_cor_ngd_df_scs_aglh',
 'rsfmri_cor_ngd_df_scs_agrh',
 'rsfmri_cor_ngd_df_scs_bs',
 'rsfmri_cor_ngd_df_scs_cdelh',
 'rsfmri_cor_ngd_df_scs_cderh',
 'rsfmri_cor_ngd_df_scs_crcxlh',
 'rsfmri_cor_ngd_df_scs_crcxrh',
 'rsfmri_cor_ngd_df_scs_hplh',
 'rsfmri_cor_ngd_df_scs_hprh',
 'rsfmri_cor_ngd_df_scs_pllh',
 'rsfmri_cor_ngd_df_scs_p

In [ ]:
print('\nPrior to merge df size is {0} and other df shape is {1}\n merging...'.format(df.shape, dat[derivative_cols].shape))
if 'src_subject_id' not in df.columns:
    # for first table assign it to df
    df = dat[derivative_cols].copy()
else:
    # all others are merged
    df = df.merge(dat[derivative_cols], how=how_merge, on=['src_subject_id', 'eventname'])
print('Any duplicated columns? {}'.format(df.columns.duplicated().any()))
print('New df size is {}'.format(df.shape))

## test dropDuplicateCols

In [27]:
df = rs.copy()
dup_pairs = [(i, j) for i,j in combinations(df, 2) if df[i].equals(df[j])]
cols_to_remove = []
for i, j in dup_pairs:
    print('{0} is identical to {1}'.format(i.upper(), j.upper()))
    cols_to_remove.append(i)

df.drop(columns=cols_to_remove, inplace=True)

RSFMRI_C_NGD_SA_NGD_DT is identical to RSFMRI_C_NGD_DT_NGD_SA
RSFMRI_C_NGD_SA_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_SA
RSFMRI_C_NGD_DT_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_DT
RSFMRI_C_NGD_AD_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_AD
RSFMRI_C_NGD_CGC_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_CGC
RSFMRI_C_NGD_CA_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_CA
RSFMRI_C_NGD_DLA_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_DLA
RSFMRI_C_NGD_FO_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_FO
RSFMRI_C_NGD_N_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_N
RSFMRI_C_NGD_RSPLTP_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_RSPLTP
RSFMRI_C_NGD_SMH_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_SMH
RSFMRI_C_NGD_SMM_NGD_VTA is identical to RSFMRI_C_NGD_VTA_NGD_SMM
RSFMRI_C_NGD_VTA_NGD_VS is identical to RSFMRI_C_NGD_VS_NGD_VTA


In [22]:
cols_to_remove

['rsfmri_c_ngd_sa_ngd_dt',
 'rsfmri_c_ngd_sa_ngd_vta',
 'rsfmri_c_ngd_dt_ngd_vta',
 'rsfmri_c_ngd_ad_ngd_vta',
 'rsfmri_c_ngd_cgc_ngd_vta',
 'rsfmri_c_ngd_ca_ngd_vta',
 'rsfmri_c_ngd_dla_ngd_vta',
 'rsfmri_c_ngd_fo_ngd_vta',
 'rsfmri_c_ngd_n_ngd_vta',
 'rsfmri_c_ngd_rspltp_ngd_vta',
 'rsfmri_c_ngd_smh_ngd_vta',
 'rsfmri_c_ngd_smm_ngd_vta',
 'rsfmri_c_ngd_vta_ngd_vs']

## test anyMRIvars function

In [73]:
df = rs.copy()

In [ ]:
# create empty list to store QC vars needed
qc_vars = []
# if any columns are found that startwith 'rsfmri_', append resting QC vars & the csv file they're found in,..
# ... creating a list of tuples which will be combined and converted to dataframe below
# this relies on the implicit boolean nature of lists, if empty their FALSE
if [col for col in df.columns if col.startswith('rsfmri_')]:
    qc_vars += [('mri_y_qc_incl', 'imgincl_rsfmri_include'), ('mri_y_qc_motion', 'rsfmri_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]

# if any columns are found that startwith 'rsfmri_', append resting QC vars & the csv file they're found in
if [col for col in df.columns if col.startswith('tfmri_nback_') or col.startswith('tfabwdp')  or col.startswith('tnbasem')]:
    qc_vars += [('mri_y_qc_incl', 'imgincl_nback_include'), ('mri_y_qc_motion', 'tfmri_nback_all_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]

# if qc_vars is NOT empty
if qc_vars:
    # remove duplicates from qc_vars
    # calling 'set' function on qc_vars returns only unique items
    # then calling 'list' function converts it back to a list
    qc_vars = list(set(qc_vars))
    # set up table_key dataframe for pulling QC vars
    # includes all QC vars selected above with cooresponding table name
    qc_key = pd.DataFrame(qc_vars, columns=['Table', 'Variable'])
    # get list of unique QC tables from qc_key tuple ex. ('mri_y_qc_incl', 'imgincl_rsfmri_include')
    qc_tables = list(set([table for table, var in qc_vars]))
    # append QC variables to dataframe
    for table in qc_tables:
        print('\nPulling from table: {0}.csv\n'.format(table))
        df = getDerivatives(df, table + '.csv', qc_key, how_merge='left')

In [75]:
qc_vars

[('mri_y_qc_incl', 'imgincl_rsfmri_include'),
 ('mri_y_qc_motion', 'rsfmri_meanmotion'),
 ('mri_y_adm_info', 'mri_info_deviceserialnumber'),
 ('mri_y_qc_incl', 'imgincl_nback_include'),
 ('mri_y_qc_motion', 'tfmri_nback_all_meanmotion'),
 ('mri_y_adm_info', 'mri_info_deviceserialnumber')]

In [76]:
qc_vars = list(set(qc_vars))
qc_vars

[('mri_y_qc_incl', 'imgincl_rsfmri_include'),
 ('mri_y_qc_motion', 'rsfmri_meanmotion'),
 ('mri_y_qc_motion', 'tfmri_nback_all_meanmotion'),
 ('mri_y_adm_info', 'mri_info_deviceserialnumber'),
 ('mri_y_qc_incl', 'imgincl_nback_include')]

In [72]:
[c for c in df.columns if 'meanmotion' in c or 'deviceserialnumber' in c or 'imgincl' in c]

['rsfmri_meanmotion',
 'tfmri_nback_all_meanmotion',
 'mri_info_deviceserialnumber',
 'imgincl_rsfmri_include',
 'imgincl_nback_include']

In [42]:
qc_key

,Table,Variable
0,mri_y_qc_incl.csv,imgincl_rsfmri_include
1,mri_y_qc_motion.csv,rsfmri_meanmotion
2,mri_y_qc_incl.csv,imgincl_nback_include
3,mri_y_adm_info.csv,mri_info_deviceserialnumber
4,mri_y_qc_motion.csv,tfmri_nback_all_meanmotion


In [33]:
qc_vars

[('mri_y_qc_incl.csv', 'imgincl_rsfmri_include'),
 ('mri_y_qc_motion.csv', 'rsfmri_meanmotion'),
 ('mri_y_adm_info.csv', 'mri_info_deviceserialnumber')]

In [27]:
qc_key

,Table,Variable
0,mri_y_qc_incl,rsfmri_meanmotion
1,mri_y_qc_incl,imgincl_rsfmri_include
2,mri_y_qc_incl,mri_info_deviceserialnumber
